In [1]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
from dgl.nn.pytorch.conv import SGConv

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lr = 0.2
bias = False
n_epochs = 100
weight_decay = 5e-4
loss_fn = torch.nn.CrossEntropyLoss()

In [3]:
data = PubmedGraphDataset()
g = data[0]
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\dgl\data\utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [4]:
model = SGConv(in_feats, n_classes, k=2, cached=True, bias=bias)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [5]:
def test(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [6]:
def train(model, g, features, labels, train_mask, val_mask):
    dur = []
    for epoch in range(n_epochs):
        model.train()
        if epoch >= 3:
            t0 = time.time()
        # forward
        logits = model(g, features)  # only compute the train set
        loss = loss_fn(logits[train_mask], labels[train_mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch >= 3:
            dur.append(time.time() - t0)

        acc = test(model, g, features, labels, val_mask)
        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f} | "
            "ETputs(KTEPS) {:.2f}". format(epoch, np.mean(dur), loss.item(),
                                            acc, n_edges / np.mean(dur) / 1000))

In [7]:
train(model, g, features, labels, train_mask, val_mask)
test(model, g, features, labels, test_mask)

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Time(s) nan | Loss 1.0998 | Accuracy 0.7260 | ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 1.0164 | Accuracy 0.7260 | ETputs(KTEPS) nan
Epoch 00002 | Time(s) nan | Loss 0.9456 | Accuracy 0.7240 | ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.0140 | Loss 0.8883 | Accuracy 0.7300 | ETputs(KTEPS) 6350.41
Epoch 00004 | Time(s) 0.0145 | Loss 0.8440 | Accuracy 0.7340 | ETputs(KTEPS) 6130.28
Epoch 00005 | Time(s) 0.0172 | Loss 0.8110 | Accuracy 0.7480 | ETputs(KTEPS) 5165.06
Epoch 00006 | Time(s) 0.0161 | Loss 0.7874 | Accuracy 0.7540 | ETputs(KTEPS) 5502.14
Epoch 00007 | Time(s) 0.0153 | Loss 0.7713 | Accuracy 0.7620 | ETputs(KTEPS) 5801.04
Epoch 00008 | Time(s) 0.0153 | Loss 0.7610 | Accuracy 0.7580 | ETputs(KTEPS) 5785.51
Epoch 00009 | Time(s) 0.0156 | Loss 0.7553 | Accuracy 0.7600 | ETputs(KTEPS) 5671.98
Epoch 00010 | Time(s) 0.0157 | Loss 0.7527 | Accuracy 0.7680 | ETputs(KTEPS) 5657.60
Epoch 00011 | Time(s) 0.0158 | Loss 0.7524 | Accuracy 0.7680 | ETputs(KTEPS) 5607.20
E

0.761